## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,26.47,97,96,3.65,overcast clouds
1,1,Shipunovo,RU,52.1582,82.2173,67.28,61,25,7.38,scattered clouds
2,2,Cape Town,ZA,-33.9258,18.4232,55.11,91,54,1.01,light rain
3,3,Qaanaaq,GL,77.4840,-69.3632,36.91,98,100,13.89,light rain
4,4,Camacha,PT,33.0833,-16.3333,67.42,73,75,9.22,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_pref_temp = float(input("What is your preferred minimum temperature for your trip? "))
max_pref_temp = float(input("What is your preferred maximum temperature for your trip? "))

What is your preferred minimum temperature for your trip? 75
What is your preferred maximum temperature for your trip? 82


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_pref_temp_df = city_data_df.loc[(city_data_df["Max Temp"]>= min_pref_temp) &
                                       (city_data_df["Max Temp"]<= max_pref_temp)
                                      ]
cities_pref_temp_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
13,13,Dwarka,IN,22.2394,68.9678,81.99,82,99,19.39,overcast clouds
22,22,Butaritari,KI,3.0707,172.7902,81.99,76,100,7.90,overcast clouds
23,23,Kayerkan,RU,69.3497,87.7602,75.52,41,28,9.35,scattered clouds
33,33,Hobyo,SO,5.3505,48.5268,75.16,78,23,20.49,few clouds
38,38,Albany,US,42.6001,-73.9662,77.41,82,48,8.81,scattered clouds
50,50,Atuona,PF,-9.8000,-139.0333,77.58,77,14,15.28,few clouds
55,55,Beidao,CN,34.5686,105.8933,77.54,56,0,3.87,clear sky
57,57,Bardiyah,LY,31.7561,25.0865,75.36,77,0,11.81,clear sky
61,61,Camocim,BR,-2.9022,-40.8411,77.07,80,62,9.66,broken clouds
70,70,Poum,NC,-20.2333,164.0167,75.60,88,69,17.07,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
cities_pref_temp_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# ******There are no null values********* Will check with the .count() method as well
cities_pref_temp_df.count()

City_ID                145
City                   145
Country                145
Lat                    145
Lng                    145
Max Temp               145
Humidity               145
Cloudiness             145
Wind Speed             145
Current Description    145
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = cities_pref_temp_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
13,Dwarka,IN,81.99,overcast clouds,22.2394,68.9678,
22,Butaritari,KI,81.99,overcast clouds,3.0707,172.7902,
23,Kayerkan,RU,75.52,scattered clouds,69.3497,87.7602,
33,Hobyo,SO,75.16,few clouds,5.3505,48.5268,
38,Albany,US,77.41,scattered clouds,42.6001,-73.9662,
50,Atuona,PF,77.58,few clouds,-9.8000,-139.0333,
55,Beidao,CN,77.54,clear sky,34.5686,105.8933,
57,Bardiyah,LY,75.36,clear sky,31.7561,25.0865,
61,Camocim,BR,77.07,broken clouds,-2.9022,-40.8411,
70,Poum,NC,75.60,broken clouds,-20.2333,164.0167,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    #Add the latitude and longitude to location key for the params dictionary
    params["location"]=f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"]=hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [11]:
#Showing count of how many items are in the hotel_df
hotel_df.count()

City                   145
Country                145
Max Temp               145
Current Description    145
Lat                    145
Lng                    145
Hotel Name             145
dtype: int64

In [12]:
#Printing the hotel_df to see if there are empty spaces under "Hotel Name"
hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
13,Dwarka,IN,81.99,overcast clouds,22.2394,68.9678,Hotel Narayan Avezika Comfort
22,Butaritari,KI,81.99,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
23,Kayerkan,RU,75.52,scattered clouds,69.3497,87.7602,
33,Hobyo,SO,75.16,few clouds,5.3505,48.5268,Iftin hotel hobyo
38,Albany,US,77.41,scattered clouds,42.6001,-73.9662,
50,Atuona,PF,77.58,few clouds,-9.8000,-139.0333,Villa Enata
55,Beidao,CN,77.54,clear sky,34.5686,105.8933,Shell Tianshui Maiji District Erma Road Walkin...
57,Bardiyah,LY,75.36,clear sky,31.7561,25.0865,بورتو بردية
61,Camocim,BR,77.07,broken clouds,-2.9022,-40.8411,Ilha Praia Hotel
70,Poum,NC,75.60,broken clouds,-20.2333,164.0167,


In [13]:
#Checking dtypes of the columns in hotel_df
hotel_df.dtypes

City                    object
Country                 object
Max Temp               float64
Current Description     object
Lat                    float64
Lng                    float64
Hotel Name              object
dtype: object

In [28]:
#Import NumPy so I can replace empty cells under "Hotel Name"
import numpy as np

#Replace empty cells under "Hotel Name" with NaN.
hotel_df["Hotel Name"].replace("", np.nan, inplace=True)


In [29]:
# 7. Drop the rows where there is no Hotel Name.
#drop NaN values
hotel_df.dropna(subset=["Hotel Name"], inplace =True)

In [30]:
hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
13,Dwarka,IN,81.99,overcast clouds,22.2394,68.9678,Hotel Narayan Avezika Comfort
22,Butaritari,KI,81.99,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
33,Hobyo,SO,75.16,few clouds,5.3505,48.5268,Iftin hotel hobyo
50,Atuona,PF,77.58,few clouds,-9.8000,-139.0333,Villa Enata
55,Beidao,CN,77.54,clear sky,34.5686,105.8933,Shell Tianshui Maiji District Erma Road Walkin...
57,Bardiyah,LY,75.36,clear sky,31.7561,25.0865,بورتو بردية
61,Camocim,BR,77.07,broken clouds,-2.9022,-40.8411,Ilha Praia Hotel
88,Montorio Al Vomano,IT,79.95,clear sky,42.5831,13.6383,Hotel Vomano
94,Mehriz,IR,81.91,clear sky,31.5917,54.4318,Mehrgard dwelling/سرای مهرگرد
96,Bandarbeyla,SO,76.75,clear sky,9.4942,50.8122,JABIR HOTEL


In [31]:
#Checking new count after removing NaNs
hotel_df.count()

City                   131
Country                131
Max Temp               131
Current Description    131
Lat                    131
Lng                    131
Hotel Name             131
dtype: int64

In [33]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [36]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [37]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))